In [1]:
from google.colab import drive
drive.mount('/drive', force_remount=True)

%cd '/drive/MyDrive/Colab Notebooks/INF2190/INF2190 Final Project/'

Mounted at /drive
/drive/MyDrive/Colab Notebooks/INF2190/INF2190 Final Project


In [2]:
import pandas as pd

billboard = pd.read_csv('data/hot-100-current.csv')
billboard['chart_week'] = billboard['chart_week'].astype('datetime64[ns]')
billboard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343600 entries, 0 to 343599
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   chart_week    343600 non-null  datetime64[ns]
 1   current_week  343600 non-null  int64         
 2   title         343600 non-null  object        
 3   performer     343600 non-null  object        
 4   last_week     311140 non-null  float64       
 5   peak_pos      343600 non-null  int64         
 6   wks_on_chart  343600 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 18.4+ MB


In [3]:
print(len(billboard))

343600


In [4]:
import datetime
billboard = billboard.loc[(billboard['chart_week'] > '1988-12-31')]
len(billboard)

184900

In [5]:
billboard_unique = pd.read_csv('data/hot-100-unique.csv')
billboard_unique = billboard_unique.drop(['tags'], axis=1)
billboard_unique.head()

,chart_week,current_week,title,performer,last_week,peak_pos,wks_on_chart
0,1989-01-07,1,Every Rose Has Its Thorn,Poison,1.0,1,11
1,1989-01-07,74,Peek-A-Boo,Siouxsie & The Banshees,63.0,53,13
2,1989-01-07,73,"Don't Worry, Be Happy (From ""Cocktail"")",Bobby McFerrin,73.0,1,24
3,1989-01-07,72,"Hippy Hippy Shake (From ""Cocktail"")",The Georgia Satellites,64.0,45,12
4,1989-01-07,71,I Beg Your Pardon,Kon Kan,87.0,71,3


In [6]:
song_title = []
artists = []

# billboard_unique = billboard

for index, row in billboard_unique.iterrows():
    duplicate = False
    title = row['title']
    artist = row['performer']

    for i in range(len(song_title)):
        if song_title[i] == title and artists[i] == artist:
            duplicate = True

    if duplicate:
        billboard_unique.drop(index, inplace=True)
    else:
        song_title.append(title)
        artists.append(artist)
        print(f"{index}: {title}, {artist}")

print(len(billboard_unique))

Streaming output truncated to the last 5000 lines.
9679: Come Get Her, Rae Sremmurd
9680: Blase, Ty Dolla $ign Featuring Future & Rae Sremmurd
9681: Runnin' (Lose It All), Naughty Boy Featuring Beyonce & Arrow Benjamin
9682: I'm The Plug, Drake & Future
9683: Die A Happy Man, Thomas Rhett
9684: 30 For 30 Freestyle, Drake
9685: Hold Each Other, A Great Big World Featuring FUTURISTIC
9686: Scholarships, Drake & Future
9687: Powerful, Major Lazer Featuring Ellie Goulding & Tarrus Riley
9688: On My Mind, Ellie Goulding
9689: Confident, Demi Lovato
9690: Diamonds Dancing, Drake & Future
9691: Digital Dash, Drake & Future
9692: Infinity, One Direction
9693: $ave Dat Money, Lil Dicky Featuring Fetty Wap & Rich Homie Quan
9694: Live From The Gutter, Drake & Future
9695: RGF Island, Fetty Wap
9696: Jumpman, Drake & Future
9697: Big Rings, Drake & Future
9698: Alive, Sia
9699: Writing's On The Wall, Sam Smith
9700: Change Locations, Drake & Future
9701: Don't, Bryson Tiller
9702: Jugg, Fetty Wap

In [7]:
song_artist_concat_list = []

for i in range(len(song_title)):
    title_artist = f"{song_title[i]} {artists[i]}"
    song_artist_concat_list.append(title_artist)

In [ ]:
# billboard_unique = billboard_unique.sort_values(by=['chart_week'])
# billboard_unique.to_csv('data/hot-100-unique.csv')

In [8]:
import os
import base64
from requests import post, get
import json
import pandas as pd
from datetime import date
from google.colab import userdata



client_id = userdata.get('client_id')
client_secret = userdata.get('client_secret')

In [25]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = (auth_string
                  .encode("utf-8"))
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token


def get_auth_header(token):
    return {"Authorization": "Bearer " + token}


def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)

    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query

    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print(f"No artists with the name {artist_name} exists...")
    else:
        return json_result


def search_for_song(token, song_name):
    url = f"https://api.spotify.com/v1/search"
    headers = get_auth_header(token)

    query = f"?q={song_name.title()}&type=track&limit=1"
    query_url = url + query

    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)["tracks"]["items"]
    if len(json_result) == 0:
        print(f"No song with the name {song_name.title()} exists...")
    else:
        return json_result


def get_track_data(token, track_id):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    return json_result


def get_song_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result


def get_audio_features(token, song, track_id):
    url = f"https://api.spotify.com/v1/audio-features"
    headers = get_auth_header(token)

    query = f"?ids={track_id}"
    query_url = url + query

    result = get(query_url, headers=headers)
    # print(result.text)
    json_result = json.loads(result.content)["audio_features"][0]
    if len(json_result) == 0:
        print(f"No song with the name {song} exists...")
    else:
        return json_result


song_data_date = []
song_name = []
song_ids = []
song_artist_name = []
song_artist_id = []
song_artist_img_url = []
song_artist_followers = []
artist_current_top_song_name = []
artist_current_top_song_id = []
album_type = []
album_name = []
album_ids = []
album_release_date = []
album_release_date_precision = []
song_isrc = []
song_len_ms = []
song_explicity = []
track_release_date = []
track_release_date_precision = []
track_img_url = []
song_acousticness = []  # Range 0.0 to 1.0 (1.0 is high confidence the track is acoustic)
song_danceability = []  # Range 0.0 to 1.0 (0.0 is least danceable and 1.0 is most danceable)
song_energy = []  # Range 0.0 to 1.0 (1.0 energetic tracks feel fast, loud, and noisy)
song_instrumentalness = []  # 0.0 to 1.0 (the closer the instrumentals value is to 1.0, the greater likelihood the track contains no vocal content.) Values above 0.5 are intended to represent instrumental tracks
song_key = []  # Integers map to pitches using https://en.wikipedia.org/wiki/Pitch_class standard notation
song_liveness = []  # Range 0.0 to 1.0 (a value above 0.8 provides strong likelihood that the track is live)
song_loudness = []  # Range -60 to 0
song_modality = []  # Major - 1 / Minor - 0
song_speechiness = []  # Range 0.0 to 1.0 (the more exclusively speech-like the recording , the closer to 1.0)
song_tempo = []  # Overall estimated tempo of a track in beats per minute (BPM)
song_time_signature = []  # The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4"
song_valence = []  # Range 0.0 to 1.0 (describing the musical positiveness conveyed by a track)

count = 1
for song in song_artist_concat_list[2700:3000]:
    today = date.today()
    song_data_date.append(today)

    token = get_token()
    result = search_for_song(token, song)

    track_name = result[0]["name"]
    song_name.append(track_name)

    song_id = result[0]["id"]
    song_ids.append(song_id)

    artist_name = result[0]["artists"][0]["name"]
    song_artist_name.append(artist_name)

    print(f"{count}/{len(song_title)}: {song} - {artist_name}")

    artist_id = result[0]["artists"][0]["id"]
    song_artist_id.append(artist_id)

    album = result[0]["album"]["album_type"]
    album_type.append(album)
    if "album" in album:
        name = result[0]["album"]["name"]
        album_uri = result[0]["album"]["uri"].split(":")[2]
        release_date = result[0]["album"]["release_date"]
        album_date_precision = result[0]["album"]["release_date_precision"]
    else:
        name = "NaN"
        album_uri = "NaN"
        release_date = "NaN"
        album_date_precision = "NaN"
    album_name.append(name)
    album_ids.append(album_uri)
    album_release_date.append(release_date)
    album_release_date_precision.append(album_date_precision)

    isrc = result[0]["external_ids"]["isrc"]
    song_isrc.append(isrc)

    song_duration = result[0]["duration_ms"]
    song_len_ms.append(song_duration)

    explicit = result[0]["explicit"]
    song_explicity.append(explicit)

    artist = search_for_artist(token, artist_name=artist_name)[0]

    try:
        artist_img_url = artist["images"][0]["url"]
    except IndexError:
        artist_img_url = "NaN"
    song_artist_img_url.append(artist_img_url)

    artist_followers = artist["followers"]["total"]
    song_artist_followers.append(artist_followers)

    songs = get_song_by_artist(token, artist_id)
    try:
        artist_current_top_song_name.append(songs[0]['name'])
        artist_current_top_song_id.append(songs[0]['id'])
    except:
        artist_current_top_song_name.append(None)
        artist_current_top_song_id.append(None)

    track = get_track_data(token, song_id)
    release_date = track["album"]["release_date"]
    track_release_date.append(release_date)

    release_date_precision = track["album"]["release_date_precision"]
    track_release_date_precision.append(release_date_precision)

    try:
        img_url = track["album"]["images"][0]["url"]
    except IndexError:
        img_url = "NaN"
    track_img_url.append(img_url)

    audio_features = get_audio_features(token, track_name, song_id)

    acousticness = audio_features["acousticness"]
    song_acousticness.append(acousticness)

    danceability = audio_features["danceability"]
    song_danceability.append(danceability)

    energy = audio_features["energy"]
    song_energy.append(energy)

    instrumentalness = audio_features["instrumentalness"]
    song_instrumentalness.append(instrumentalness)

    key = audio_features["key"]
    song_key.append(key)

    liveness = audio_features["liveness"]
    song_liveness.append(liveness)

    loudness = audio_features["loudness"]
    song_loudness.append(loudness)

    modality = audio_features["mode"]
    song_modality.append(modality)

    speechiness = audio_features["speechiness"]
    song_speechiness.append(speechiness)

    tempo = audio_features["tempo"]
    song_tempo.append(tempo)

    time_signature = audio_features["time_signature"]
    song_time_signature.append(time_signature)

    valence = audio_features["valence"]
    song_valence.append(valence)

    count += 1


1/14678: Ain't Nobody/Kissing You Faith Evans - Faith Evans
2/14678: Machinehead (From "Fear") Bush - Bush
3/14678: Drop The Pharcyde - The Pharcyde
4/14678: Guilty (From "Seven") Gravity Kills - Pixel Terror
5/14678: Don't Wanna Lose You Lionel Richie - Lionel Richie
6/14678: Reach Gloria Estefan - Gloria Estefan
7/14678: You're The One SWV - SWV
8/14678: Chains Tina Arena - Tina Arena
9/14678: Slow Jams Quincy Jones Feat. Babyface & Tamia With Portrai - Quincy Jones
10/14678: Love Don't Live Here Anymore Madonna - Madonna
11/14678: Live And Die For Hip Hop Kris Kross - Kris Kross
12/14678: My Maria Brooks & Dunn - Brooks & Dunn
13/14678: No More Games Skin Deep Featuring Li'l Kim Of Junior M.A.F.I.A. - Junior M.A.F.I.A.
14/14678: Scarred (From "Eddie") Luke - Eddie Flint
15/14678: Tres Delinquentes Delinquent Habits - Delinquent Habits
16/14678: Mr. Ice Cream Man Master P - Master P
17/14678: Old Man & Me (When I Get To Heaven) Hootie & The Blowfish - Ryan Harvey
18/14678: Feel The M

KeyError: 'tracks'

In [ ]:
print(len(song_explicity))
print(8776+205)
print(len(song_title[5500:5907]))

268
8981
407


In [ ]:
spotify_song_data = pd.DataFrame({"data_date": song_data_date,
                                  "billboard_song_title": song_title[500:1000],
                                  "billboard_song_artist": artists[500:1000],
                                  "song_name": song_name,
                                  "song_id": song_ids,
                                  "song_artist_name": song_artist_name,
                                  "song_artist_id": song_artist_id,
                                  "song_artist_img_url": song_artist_img_url,
                                  "song_artist_followers": song_artist_followers,
                                  "artist_current_top_song_id": artist_current_top_song_id,
                                  "artist_current_top_song_name": artist_current_top_song_name,
                                  "album_type": album_type,
                                  "album_name": album_name,
                                  "album_ids": album_ids,
                                  "album_release_date": album_release_date,
                                  "album_release_date_precision": album_release_date_precision,
                                  "song_isrc": song_isrc,
                                  "song_len_ms": song_len_ms,
                                  "song_explicity": song_explicity,
                                  "track_release_date": track_release_date,
                                  "track_release_date_precision": track_release_date_precision,
                                  "track_img_url": track_img_url,
                                  "song_acousticness": song_acousticness,
                                  "song_danceability": song_danceability,
                                  "song_energy": song_energy,
                                  "song_instrumentalness": song_instrumentalness,
                                  "song_key": song_key,
                                  "song_liveness": song_liveness,
                                  "song_loudness": song_loudness,
                                  "song_modality": song_modality,
                                  "song_speechiness": song_speechiness,
                                  "song_tempo": song_tempo,
                                  "song_time_signature": song_time_signature,
                                  "song_valence": song_valence
                                  })

spotify_song_data.to_csv(f"data/spotify_api_data/spotify_api_data_{today}_1000.csv", index=False)

In [24]:
length = 53
beginning = 2700
end = beginning + length

spotify_song_data = pd.DataFrame({"data_date": song_data_date[:length],
                                  "billboard_song_title": song_title[beginning:end],
                                  "billboard_song_artist": artists[beginning:end],
                                  "song_name": song_name[:length],
                                  "song_id": song_ids[:length],
                                  "song_artist_name": song_artist_name[:length],
                                  "song_artist_id": song_artist_id[:length],
                                  "song_artist_img_url": song_artist_img_url[:length],
                                  "song_artist_followers": song_artist_followers[:length],
                                  "artist_current_top_song_id": artist_current_top_song_id[:length],
                                  "artist_current_top_song_name": artist_current_top_song_name[:length],
                                  "album_type": album_type[:length],
                                  "album_name": album_name[:length],
                                  "album_ids": album_ids[:length],
                                  "album_release_date": album_release_date[:length],
                                  "album_release_date_precision": album_release_date_precision[:length],
                                  "song_isrc": song_isrc[:length],
                                  "song_len_ms": song_len_ms[:length],
                                  "song_explicity": song_explicity[:length],
                                  "track_release_date": track_release_date[:length],
                                  "track_release_date_precision": track_release_date_precision[:length],
                                  "track_img_url": track_img_url[:length],
                                  "song_acousticness": song_acousticness[:length],
                                  "song_danceability": song_danceability[:length],
                                  "song_energy": song_energy[:length],
                                  "song_instrumentalness": song_instrumentalness[:length],
                                  "song_key": song_key[:length],
                                  "song_liveness": song_liveness[:length],
                                  "song_loudness": song_loudness[:length],
                                  "song_modality": song_modality[:length],
                                  "song_speechiness": song_speechiness[:length],
                                  "song_tempo": song_tempo[:length],
                                  "song_time_signature": song_time_signature[:length],
                                  "song_valence": song_valence[:length]
                                  })

spotify_song_data.to_csv(f"data/spotify_api_data_{today}_{end}.csv", index=False)
print(f'start: {beginning}, end: {end}')

start: 2647, end: 2700
